Name: Shuyun Wen

Email: wendywen027@gmail.com

#NOTE:
Please insert a text cell at the top of the Notebook with your Full Name in Bold and your email that you use for both Journey and Github

Also, add to the title of the notebook, an underscore as well as your name.

Ex. 'Workshop_Hypothesis_Testing_Usama'

#Warmup

Import pandas, numpy, scipy.stats, statsmodels.api & statsmodels.formula.api.ols 

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as sc
import statsmodels.api as sm
from statsmodels.formula.api import ols

#Hypothesis Testing

##Iris Data
It is believed that the average sepal length of the setosa species is 4 cm. Use the Iris data set as well as the appropriate test to determine if we have evidence to reject this claim.

In [3]:
url = "https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv"
df = pd.read_csv(url)

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


Use an appropriate test to see if there is a significant difference in the average sepal width between the versicolor and virginica species.

In [4]:
grp1 = np.array(df[df["species"]=='versicolor']['sepal_width'])
grp2 = np.array(df[df["species"]=='virginica']['sepal_width'])

sc.ttest_ind(grp1, grp2, equal_var=False)

Ttest_indResult(statistic=-3.2057607502218186, pvalue=0.001819483482104968)

Null Hypothesis: there is no significant difference in sepal width between the versicolor and virginica species.

Test result: with pvalue = 0.002, we can reject the null hypothesis 99% level of confidence.Thus, there is a significant difference in the average sepal width between the versicolor and virginica species.


##Robberies in Toronto

Download the Robberies in Toronto Data Set from this link (https://data.torontopolice.on.ca/datasets/robbery-2014-to-2019?geometry=-79.896%2C43.627%2C-78.853%2C43.800) and read it into a DataFrame. Use an appropriate test to see if whether the Robbery occured on a weekend affected whether the Robbery would occur in the Day or in the Night. 

**HINT:** You may assume that day hours are from 6 am to 6 pm. Before carrying out your analysis, you must modify your data frame such that each row has a column saying whether the crime occured over the weekend or not as well as a column saying whether the crime occured during the day or night.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [26]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Robbery.csv']))

In [33]:
features = ['event_unique_id','occurrencedayofweek','occurrencehour']
df1 = df.loc[:,features]
df1.head()

,event_unique_id,occurrencedayofweek,occurrencehour
0,GO-2018779070,Monday,19
1,GO-2018812524,Saturday,23
2,GO-2018824134,Friday,17
3,GO-2018871791,Monday,20
4,GO-2018871791,Monday,20


In [34]:
df1.loc[df1['occurrencedayofweek']=='Saturday','occurrencedayofweek'] = 'weekend'
df1.loc[df1['occurrencedayofweek']=='Sunday','occurrencedayofweek'] = 'weekend'
df1.loc[df1['occurrencedayofweek']!='weekend','occurrencedayofweek'] = 'week'
df1.loc[(df1['occurrencehour']<6)|(df1['occurrencehour']>18),'occurrencehour'] = 'night'
df1.loc[df1['occurrencehour']!='night','occurrencehour'] = 'day'

df1.columns=['id','week','day']

df1.head()

,id,week,day
0,GO-2018779070,week,night
1,GO-2018812524,weekend,night
2,GO-2018824134,week,day
3,GO-2018871791,week,night
4,GO-2018871791,week,night


In [35]:
week_day = pd.crosstab(index=df1['week'],columns = df1['day'])
week_day

day,day,night
week,,
week,8510,9296
weekend,2509,4647


In [38]:
(chi2, p, dof, _) = sc.chi2_contingency([week_day.iloc[0].values, week_day.iloc[1].values])
print(f"The Chi Square value is {chi2} with a p value of {p}")

The Chi Square value is 335.04328643964504 with a p value of 7.662626380973767e-75


According to chi square test result(p value of 7.662626380973767e-75), we could reject null hypothesis with 99% confidence level that the robbery occured on a weekend or not affects the occurance hour(Day/Night) of robbery.  

##Stroke Prediction

Load the following dataset off Kaggle: https://www.kaggle.com/fedesoriano/stroke-prediction-dataset

We are interested in creating a model that can predict someone's BMI based on certain features. Use the appropriate test to determine if Hypertension and whether someone has been married or not affects their BMI.

### data load

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d fedesoriano/stroke-prediction-dataset

In [9]:
from zipfile import ZipFile
file_name = "stroke-prediction-dataset.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [10]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')

### test

In [ ]:
df = df.loc[:,['gender','ever_married','bmi']]
df = df.dropna()
df.head()

In [12]:
model = ols('bmi ~ C(gender) + C(ever_married) + C(ever_married):C(gender)', data = df).fit()
sm.stats.anova_lm(model, typ=2)

/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  'rank is %d' % (J, J_), ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1768: RuntimeWarning: invalid value encountered in true_divide
  F /= J
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  'rank is %d' % (J, J_), ValueWarning)


,sum_sq,df,F,PR(>F)
C(gender),36.694796,2.0,0.337195,0.561479
C(ever_married),NaN,1.0,NaN,NaN
C(ever_married):C(gender),1015.740052,2.0,9.333829,0.002262
Residual,266835.242671,4904.0,NaN,NaN


According to the result table, marrige status and combination of two categories pass Anova test, which means we can reject null hypothesis with 99% level of confidence (pvalue < 0.01), and they have statistic significance on bmi value. However, 'gender' itself doesn't pass Anova test with pvalue=0.56, thus we can not reject null hypothesis, which is "gender has no significant influence on bmi value".